In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [2]:
url='https://chipublib.bibliocommons.com/events/rss/all'
program_list={}
my_dict = {"program_name":[],
           "program_description":[],
           "program_url":[],
           "virtual":[],
           "registration_url":[],
           "start_date":[],
           "end_date":[],
           "program":[],
           "category":[],
          "audience":[],
          "language":[],
          "location_name":[],
          "location_address":[],
          "location_city":[],
          "location_state":[],
          "location_zip":[],
          "contact_name":[],
          "contact_phone":[],
          "contact_email":[]}

In [3]:
r=requests.get(url)
soup=bs(r.text, features='xml')
#         return print('CPL scrape successful: ',r.status_code)
programs=soup.find_all('item')

In [4]:
for p in programs:
            title=p.find('title').text
            description=p.find('description').text.strip()
            link=p.find('link').text
            virtual=p.find('bc:is_virtual').text
            start_date=p.find('bc:start_date_local').text
            end_date=p.find('bc:end_date_local').text
            program=getattr(p.find('category',domain='Program'), 'text', 'null')
            category=getattr(p.find('category',domain='Type'), 'text', 'null')
            audience=p.find_all('category',domain='Audience')
            language=p.find_all('category',domain='Language')
            location=p.find_all('bc:location')
            if location is not None:
                for l in location:
                    location_name=getattr(l.find('bc:name'), 'string', None)
                
            location_number=getattr(p.find('bc:number'), 'string', 'null')
            location_street=getattr(p.find('bc:street'), 'string', 'null')
            if location_number!=location_street:
                location_address=f'{location_number} {location_street}'
            else:
                location_address=location_number
            
            location_city=getattr(p.find('bc:city'), 'string', 'null')
            location_zip=getattr(p.find('bc:zip'), 'string', 'null')
            location_state=getattr(p.find('bc:state'), 'string', 'null')
            contact_name=p.find('bc:contact')
            contact_phone=getattr(p.find('bc:phone'), 'string', 'null')
            contact_email=getattr(p.find('bc:email'), 'string', 'null')
            
            
            my_dict["program_name"].append(title)
            my_dict["program_description"].append(description)
            my_dict["program_url"].append(link)
            my_dict["registration_url"].append(link)
            my_dict["virtual"].append(virtual)
            my_dict["start_date"].append(start_date)
            my_dict["end_date"].append(start_date)
            my_dict["program"].append(program)
            my_dict["category"].append(category)
            my_dict["audience"].append(audience)
            my_dict["language"].append(language)
            my_dict["location_name"].append(location_name)
            my_dict["location_address"].append(location_address)
            my_dict["location_city"].append(location_city)
            my_dict["location_state"].append(location_state)
            my_dict["location_zip"].append(location_zip)
            my_dict["contact_name"].append(contact_name)
            my_dict["contact_phone"].append(contact_phone)
            my_dict["contact_email"].append(contact_email)
            
#             program={'title':title,
#                     'link':link,
#                     'start date':start_date
#                     }
#             program_list.append(program)

In [5]:
df = pd.DataFrame(my_dict)
df.head(30)

,program_name,program_description,program_url,virtual,registration_url,start_date,end_date,program,category,audience,language,location_name,location_address,location_city,location_state,location_zip,contact_name,contact_phone,contact_email
0,Grab and Go: Paper Rose Craft,<p>In celebration of Latinx History Month come...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-07T10:00,2021-10-07T10:00,Latinx History,"Crafts, Games and Play",[[Kids]],[[English]],Manning,6 S. Hoyne Avenue,Chicago,IL,60612,"[[Ms. Jade], [+1-312-746-6800], [jnewman@chipu...",+1-312-746-6800,jnewman@chipublib.org
1,Toddler Time,<p>Join the staff of the Thomas Hughes Childre...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-07T10:00,2021-10-07T10:00,null,Story Time,[[Toddlers]],[[English]],Manning,null,null,null,null,"[[Thomas Hughes Children's Library], [+1-312-7...",+1-312-747-4200,null
2,LOWELL CLASS VISIT & STORY TIME,<p>Class Visit from&nbsp;Lowell Elementary&nbs...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-07T10:15,2021-10-07T10:15,null,Outreach,[[Kids]],"[[English], [Español]]",Humboldt Park,1605 N. Troy Street,Chicago,IL,60647,[[Aldo Vasquez]],null,null
3,Book Zoomies,"<p>Book Zoomies presents funny books, silly so...",https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-07T10:30,2021-10-07T10:30,null,Story Time,"[[Babies], [Preschoolers], [Toddlers]]",[[English]],Humboldt Park,null,null,null,null,"[[Miss Nancy], [+1-312-747-5500]]",+1-312-747-5500,null
4,The Art of Storytelling with Pixar: In Person ...,<p><strong>THE ART </strong>OF STORYTELLING LE...,https://chipublib.bibliocommons.com/events/613...,false,https://chipublib.bibliocommons.com/events/613...,2021-10-07T10:30,2021-10-07T10:30,"One Book, One Chicago",Continuing Education and College,[[Adults]],[[English]],Toman,2708 S. Pulaski Road,Chicago,IL,60623,[[Portia ]],null,null
5,Preschool Story Time,<p>Ms. April from the Blackstone Branch will s...,https://chipublib.bibliocommons.com/events/613...,false,https://chipublib.bibliocommons.com/events/613...,2021-10-07T10:30,2021-10-07T10:30,null,Story Time,[[Preschoolers]],[[English]],Blackstone,4904 S. Lake Park Avenue,Chicago,IL,60615,"[[April Lundberg], [+1-312-747-0511], [alundbe...",+1-312-747-0511,alundberg@chipublib.org
6,Story Time with Ms. Jessie,<p>Join Ms. Jessie of the Rogers Park Branch f...,https://chipublib.bibliocommons.com/events/610...,true,https://chipublib.bibliocommons.com/events/610...,2021-10-07T10:30,2021-10-07T10:30,null,Story Time,"[[Babies], [Preschoolers], [Toddlers]]",[[English]],Blackstone,null,null,null,null,"[[Jessie], [+1-312-744-0156], [jwehrle@chipubl...",+1-312-744-0156,jwehrle@chipublib.org
7,Preschool Story Time with Budlong Woods Branch,"<p>Join us for fun stories, songs&nbsp;and a c...",https://chipublib.bibliocommons.com/events/614...,true,https://chipublib.bibliocommons.com/events/614...,2021-10-07T10:30,2021-10-07T10:30,null,Story Time,[[Preschoolers]],[[English]],Blackstone,null,null,null,null,"[[Children's Librarian], [budlongwoods@chipubl...",null,budlongwoods@chipublib.org
8,Fall Story Time with Ms. Sharon,<p>Join Ms. Sharon from the&nbsp;Dunning Branc...,https://chipublib.bibliocommons.com/events/613...,true,https://chipublib.bibliocommons.com/events/613...,2021-10-07T10:45,2021-10-07T10:45,null,Story Time,"[[Babies], [Preschoolers], [Toddlers]]",[[English]],Blackstone,null,null,null,null,"[[Sharon Anderson], [+1-312-743-0480], [sander...",+1-312-743-0480,sanderso@chipublib.org
9,Research Your Family History with the Polish G...,"<figure><a href=""https://pgsa.org/"" target=""_b...",https://chipublib.bibliocommons.com/events/614...,true,https://chipublib.bibliocommons.com/events/614...,2021-10-07T11:00,2021-10-07T11:00,Polish American Heritag

In [6]:
df=df.astype({"language":'string',
             "audience":'string',
             "contact_name":'string'})

In [7]:
df['location_name'] = np.where(df["virtual"]=='true', 'null', df['location_name'])
df['meeting_type'] = np.where(df["virtual"]=='true', 'online', 'face_to_face')
df[['start_date','start_time']] = df['start_date'].str.split('T',expand=True)
df[['end_date','end_time']] = df['end_date'].str.split('T',expand=True)

In [8]:
df["contact_name"]=df["contact_name"].str.split(']',1,expand=True)
df['contact_name'] = df['contact_name'].str.replace(r'<[^<>]*>', '', regex=True)
df['contact_name'] = df['contact_name'].str.replace('[','')
df['contact_name'] = df['contact_name'].str.replace(']','')
df["contact_name"]=df["contact_name"].str.split('+',1,expand=True)

In [9]:
df['contact_phone'] = df['contact_phone'].str.replace('+','')
df['contact_phone'] = df['contact_phone'].str.replace('1-','')
df['contact_phone'] = df['contact_phone'].str.replace('-','')

In [10]:
df

,program_name,program_description,program_url,virtual,registration_url,start_date,end_date,program,category,audience,...,location_address,location_city,location_state,location_zip,contact_name,contact_phone,contact_email,meeting_type,start_time,end_time
0,Grab and Go: Paper Rose Craft,<p>In celebration of Latinx History Month come...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-07,2021-10-07,Latinx History,"Crafts, Games and Play","[<category domain=""Audience"">Kids</category>]",...,6 S. Hoyne Avenue,Chicago,IL,60612,Ms. Jade,3127466800,jnewman@chipublib.org,face_to_face,10:00,10:00
1,Toddler Time,<p>Join the staff of the Thomas Hughes Childre...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-07,2021-10-07,null,Story Time,"[<category domain=""Audience"">Toddlers</category>]",...,null,null,null,null,Thomas Hughes Children's Library,3127474200,null,online,10:00,10:00
2,LOWELL CLASS VISIT & STORY TIME,<p>Class Visit from&nbsp;Lowell Elementary&nbs...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-07,2021-10-07,null,Outreach,"[<category domain=""Audience"">Kids</category>]",...,1605 N. Troy Street,Chicago,IL,60647,Aldo Vasquez,null,null,face_to_face,10:15,10:15
3,Book Zoomies,"<p>Book Zoomies presents funny books, silly so...",https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-07,2021-10-07,null,Story Time,"[<category domain=""Audience"">Babies</category>...",...,null,null,null,null,Miss Nancy,3127475500,null,online,10:30,10:30
4,The Art of Storytelling with Pixar: In Person ...,<p><strong>THE ART </strong>OF STORYTELLING LE...,https://chipublib.bibliocommons.com/events/613...,false,https://chipublib.bibliocommons.com/events/613...,2021-10-07,2021-10-07,"One Book, One Chicago",Continuing Education and College,"[<category domain=""Audience"">Adults</category>]",...,2708 S. Pulaski Road,Chicago,IL,60623,Portia,null,null,face_to_face,10:30,10:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,Books 'n Babes,"<p>Come one, come all, to our fun story time f...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-02-28,2022-02-28,null,Story Time,"[<category domain=""Audience"">Babies</category>]",...,null,null,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,10:30,10:30
1335,Books 'n Babes,"<p>Come one, come all, to our fun story time f...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-03-14,2022-03-14,null,Story Time,"[<category domain=""Audience"">Babies</category>]",...,null,null,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,10:30,10:30
1336,Books 'n Babes,"<p>Come one, come all, to our fun story time f...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-03-28,2022-03-28,null,Story Time,"[<category domain=""Audience"">Babies</category>]",...,null,null,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,10:30,10:30
1337,"Law at the Library: Wills, Trusts and Estate P...","<figure><img src=""https://chipublib.bibliocomm...",https://chipublib.bibliocommons.com/events/611...,true,https://chipublib.bibliocommons.com/events/611...,2022-03-29,2022-03-29,Law at the Library,"Business, Law and Money","[<category domain=""Audience"">Adults</category>]",...,null,null,null,null,adultservices@chipublib.org,null,adultservices@chipublib.org,online,18:00,18:00


In [11]:
# def cpl_programs(url):
#     program_list=[]
#     try:
#         r=requests.get(url)
#         soup=BeautifulSoup(r.content, features='xml')
# #         return print('CPL scrape successful: ',r.status_code)
#         programs = soup.findAll('item')
    
#         for p in programs:
#             title=p.find('title').text
#             link=p.find('link').text
#             start_date=p.find('bc:start_date_local').text
            
#             program={'title':title,
#                     'link':link,
#                     'start date':start_date
#                     }
#             program_list.append(program)
        

#         return print(program_list)
#     except Exception as e:
#         print('FAILED. See exception: ')
#         print(e)
        


In [12]:
# print('Starting scrape')
# cpl_programs(url)
# print('Finished')

In [13]:
# df['category'] = df['category'].str.replace('Crafts, Games and Play', '1')
# df['category'] = df['category'].str.replace('Story Time', '2')
# df['category'] = df['category'].str.replace('Continuing Education and College', '3')
# df['category'] = df['category'].str.replace('Celebrations', '4')
# df['category'] = df['category'].str.replace('Jobs and Careers', '5')
# df['category'] = df['category'].str.replace('Book Clubs', '6')
# df['category'] = df['category'].str.replace('Computers and Technology', '7')

In [14]:
cat_list=df['category'].unique()

In [15]:
cat_list

array(['Crafts, Games and Play', 'Story Time', 'Outreach',
       'Continuing Education and College', 'Celebrations',
       'Jobs and Careers', 'Book Clubs', 'Computers and Technology',
       'Health and Science', 'Workshops', 'Art, Movies and Performances',
       'Lectures', 'Writing and Poetry', 'Business, Law and Money', 'DIY',
       'Tours', 'History and Genealogy', 'Author Events',
       'Library Closures'], dtype=object)

In [16]:
len(cat_list)

19

In [17]:
#MCMF_list=list(range(1,len(cat_list)+1))

In [18]:
MCMF_list=[232,230,223,232,222,230,228,218,232,232,223,230,224,210,215,221,230,215]

In [19]:
MCMF_list=list(map(str, MCMF_list))

In [20]:
ids = {'lib_cats':cat_list,
      'MCMF_cat':MCMF_list}

ids = dict(zip(ids['lib_cats'], ids['MCMF_cat']))

In [21]:
for old, new in ids.items():
    df['category'] = df['category'].str.replace(old, new, regex=False)

In [22]:
df['language'] = df['language'].str.replace(r'<[^<>]*>', '', regex=True)
df['language'] = df['language'].str.replace('[','')
df['language'] = df['language'].str.replace(']','')

In [23]:
df['audience'] = df['audience'].str.replace(r'<[^<>]*>', '', regex=True)
df['audience'] = df['audience'].str.replace('[','')
df['audience'] = df['audience'].str.replace(']','')
df['audience_max']=df['audience']

In [24]:
df['program_description'] = df['program_description'].str.replace(r'<[^<>]*>', '', regex=True)

In [25]:
df

,program_name,program_description,program_url,virtual,registration_url,start_date,end_date,program,category,audience,...,location_city,location_state,location_zip,contact_name,contact_phone,contact_email,meeting_type,start_time,end_time,audience_max
0,Grab and Go: Paper Rose Craft,In celebration of Latinx History Month come&nb...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-07,2021-10-07,Latinx History,232,Kids,...,Chicago,IL,60612,Ms. Jade,3127466800,jnewman@chipublib.org,face_to_face,10:00,10:00,Kids
1,Toddler Time,Join the staff of the Thomas Hughes Children's...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-07,2021-10-07,null,230,Toddlers,...,null,null,null,Thomas Hughes Children's Library,3127474200,null,online,10:00,10:00,Toddlers
2,LOWELL CLASS VISIT & STORY TIME,Class Visit from&nbsp;Lowell Elementary&nbsp;1...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-07,2021-10-07,null,223,Kids,...,Chicago,IL,60647,Aldo Vasquez,null,null,face_to_face,10:15,10:15,Kids
3,Book Zoomies,"Book Zoomies presents funny books, silly songs...",https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-07,2021-10-07,null,230,"Babies, Preschoolers, Toddlers",...,null,null,null,Miss Nancy,3127475500,null,online,10:30,10:30,"Babies, Preschoolers, Toddlers"
4,The Art of Storytelling with Pixar: In Person ...,THE ART OF STORYTELLING LEARNING CIRCLES How w...,https://chipublib.bibliocommons.com/events/613...,false,https://chipublib.bibliocommons.com/events/613...,2021-10-07,2021-10-07,"One Book, One Chicago",232,Adults,...,Chicago,IL,60623,Portia,null,null,face_to_face,10:30,10:30,Adults
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334,Books 'n Babes,"Come one, come all, to our fun story time for ...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-02-28,2022-02-28,null,230,Babies,...,null,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,10:30,10:30,Babies
1335,Books 'n Babes,"Come one, come all, to our fun story time for ...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-03-14,2022-03-14,null,230,Babies,...,null,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,10:30,10:30,Babies
1336,Books 'n Babes,"Come one, come all, to our fun story time for ...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-03-28,2022-03-28,null,230,Babies,...,null,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,10:30,10:30,Babies
1337,"Law at the Library: Wills, Trusts and Estate P...",Each Law at the Library program features a pre...,https://chipublib.bibliocommons.com/events/611...,true,https://chipublib.bibliocommons.com/events/611...,2022-03-29,2022-03-29,Law at the Library,210,Adults,...,null,null,null,adultservices@chipublib.org,null,adultservices@chipublib.org,online,18:00,18:00,Adults


In [26]:
df.to_csv('test_run.csv')

In [30]:
aud_list=['Babies','Toddlers','Preschoolers','Kids','Tweens','Teens','Adults','Seniors','All Ages']

In [31]:
babymin,babymax=0,2
babyrange=list(range(babymin, babymax+1))
toddlermin,toddlermax=1,3
toddlerrange=list(range(toddlermin, toddlermax+1))
prekmin,prekmax=2,4
prekrange=list(range(prekmin, prekmax+1))
kidmin,kidmax=4,10
kidrange=list(range(kidmin, kidmax+1))
tweenmin,tweenmax=9,12
tweenrange=list(range(tweenmin, tweenmax+1))
teenmin,teenmax=13,18
teenrange=list(range(teenmin, teenmax+1))
adultmin,adultmax=18,99
adultrange=list(range(adultmin, adultmax+1))
seniormin,seniormax=65,99
seniorrange=list(range(seniormin, seniormax+1))
allmin,allmax=0,99
allrange=list(range(allmin, allmax+1))

In [32]:
aud_min=[babymin,toddlermin,prekmin,kidmin,tweenmin,teenmin,adultmin,seniormin,allmin]
aud_min=list(map(str, aud_min))
aud_max=[babymax,toddlermax,prekmax,kidmax,tweenmax,teenmax,adultmax,seniormax,allmax]
aud_max=list(map(str, aud_max))

In [33]:
audhelpmin = {'aud_list':aud_list,
      'aud_min':aud_min}

audhelpmin = dict(zip(audhelpmin['aud_list'], audhelpmin['aud_min']))

In [34]:
for old, new in audhelpmin.items():
    df['audience'] = df['audience'].str.replace(old, new, regex=False)

In [35]:
df['audience'] = df['audience'].str.replace(' ','')

In [36]:
minlist=[]
for x in df['audience'].str.split(','):
    minlist.append(min(x))
    
df['min_age']=minlist

In [37]:
audhelpmax = {'aud_list':aud_list,
      'aud_max':aud_max}

audhelpmax = dict(zip(audhelpmax['aud_list'], audhelpmax['aud_max']))

In [38]:
for old, new in audhelpmax.items():
    df['audience_max'] = df['audience_max'].str.replace(old, new, regex=False)

In [39]:
df['audience_max'] = df['audience_max'].str.replace(' ','')

In [40]:
maxlist=[]
for x in df['audience_max'].str.split(','):
    maxlist.append(max(x))
    
df['max_age']=maxlist

In [42]:
df.to_csv('test_run.csv')